# Create EDD Study Files From Data

This notebook creates the files needed for importing a study into Experiment Data Depot (EDD).

## Inputs and outputs

### Required file to run this notebook:
   - `../flaviolin yield data/DBTL2/media_descriptions.csv` - media designs for each of the wells
   
   - `../flaviolin yield data/DBTL2/OD.xlsx` - production data from the plate reader


### File generated by running this notebook:
   - `edd_experiment_description.csv`
   
   - `edd_protocol.csv`
 
    
The files are stored in the user defined directory.

## Setup

Importing needed libraries:

In [1]:
import sys
sys.path.append('../media_compiler')

import pandas as pd
import openpyxl

from core import create_media_description

### User parameters

In [2]:
CYCLE = 2

user_params = {
    'media_file': f'../flaviolin yield data/DBTL{CYCLE}/media_descriptions.csv',  
    'measurement_file': f'../flaviolin yield data/DBTL{CYCLE}/OD.xlsx',
    'output_file_path': f'../flaviolin yield data/DBTL{CYCLE}', # Folder for output files,
    'num_replicates': 3,
    'num_designs': 16,
    'protocol_name': ['OD600', 'OD340'],
    'time_point': 48,
    'part_id': 'JBx_193086',
    'media': 'MOPS',
    'culture_volume': 15,
    'well_volume': 1500,
    'shaking_speed': 800,
    'temperature': 30,
    } 


In [3]:
df = pd.read_csv(user_params['media_file'], index_col=0)
df.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l]
Well,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.003982,147.780153,0.263819,1.292187,0.001388,10.224859,1.888788,146.589727,0.000013,0.001232,0.0001,0.0008,0.0001,0.05
B1,40.0,4.0,0.003982,147.780153,0.263819,1.292187,0.001388,10.224859,1.888788,146.589727,0.000013,0.001232,0.0001,0.0008,0.0001,0.05
C1,40.0,4.0,0.003982,147.780153,0.263819,1.292187,0.001388,10.224859,1.888788,146.589727,0.000013,0.001232,0.0001,0.0008,0.0001,0.05
D1,40.0,4.0,0.000835,168.691467,1.582973,5.928974,0.008197,12.175665,2.941506,93.463208,0.000219,0.000281,0.0001,0.0008,0.0001,0.05
E1,40.0,4.0,0.000835,168.691467,1.582973,5.928974,0.008197,12.175665,2.941506,93.463208,0.000219,0.000281,0.0001,0.0008,0.0001,0.05


## Create Line Description

In [4]:
df['Line Description'] = df.apply(create_media_description, axis=1)


In [5]:
df['Line Description']

Well
A1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F3    MOPS[mM]: 40.000000, Tricine[mM]: 4.0

## Create Line Names

Add metadata for media and replicates to craft Line Names as **C[.]\_W[.]1\_[.]3-R[.]** denoting cycle number, wells occupying the same design and replicate number:

Check if it's row or column order of lines:

In [6]:
column_order = True if df.index[1][0] == 'B' else False

In [7]:
reps = user_params['num_replicates']
num_media_designs = user_params['num_designs']

def linefunction_colum_order(row):
    well = row.name
    if well[0] in 'ABC':
        return f'C{CYCLE}_WA{well[1]}_C{well[1]}-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_WD{well[1]}_F{well[1]}-R{row["Replicate"]}'                                                                                              

def linefunction_row_order(row):
    well = row.name
    if int(well[1]) < 5:
        return f'C{CYCLE}_W{well[0]}1_{well[0]}4-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_W{well[0]}5_{well[0]}8-R{row["Replicate"]}'      
    
def linefunction_row_order_triplicates(row):
    well = row.name

    well_names = list(df.index)
    index = well_names.index(well)
    rem = index%3
    name = f'C{CYCLE}_W{well_names[index-rem]}_{well_names[index-rem+2]}-R{row["Replicate"]}'
    print(name)
    return name
    
    

    
df['Replicate'] = [i+1 for _ in range(num_media_designs) for i in range(reps)]
df['Line Name'] = df.apply(linefunction_row_order_triplicates, axis=1)

# if column_order:
#     df['Line Name'] = df.apply(linefunction_colum_order, axis=1)
# else:
#     df['Line Name'] = df.apply(linefunction_row_order, axis=1)

C2_WA1_C1-R1
C2_WA1_C1-R2
C2_WA1_C1-R3
C2_WD1_F1-R1
C2_WD1_F1-R2
C2_WD1_F1-R3
C2_WA2_C2-R1
C2_WA2_C2-R2
C2_WA2_C2-R3
C2_WD2_F2-R1
C2_WD2_F2-R2
C2_WD2_F2-R3
C2_WA3_C3-R1
C2_WA3_C3-R2
C2_WA3_C3-R3
C2_WD3_F3-R1
C2_WD3_F3-R2
C2_WD3_F3-R3
C2_WA4_C4-R1
C2_WA4_C4-R2
C2_WA4_C4-R3
C2_WD4_F4-R1
C2_WD4_F4-R2
C2_WD4_F4-R3
C2_WA5_C5-R1
C2_WA5_C5-R2
C2_WA5_C5-R3
C2_WD5_F5-R1
C2_WD5_F5-R2
C2_WD5_F5-R3
C2_WA6_C6-R1
C2_WA6_C6-R2
C2_WA6_C6-R3
C2_WD6_F6-R1
C2_WD6_F6-R2
C2_WD6_F6-R3
C2_WA7_C7-R1
C2_WA7_C7-R2
C2_WA7_C7-R3
C2_WD7_F7-R1
C2_WD7_F7-R2
C2_WD7_F7-R3
C2_WA8_C8-R1
C2_WA8_C8-R2
C2_WA8_C8-R3
C2_WD8_F8-R1
C2_WD8_F8-R2
C2_WD8_F8-R3


In [8]:
df

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name
Well,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.003982,147.780153,0.263819,1.292187,0.001388,10.224859,1.888788,146.589727,0.000013,0.001232,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C2_WA1_C1-R1
B1,40.0,4.0,0.003982,147.780153,0.263819,1.292187,0.001388,10.224859,1.888788,146.589727,0.000013,0.001232,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C2_WA1_C1-R2
C1,40.0,4.0,0.003982,147.780153,0.263819,1.292187,0.001388,10.224859,1.888788,146.589727,0.000013,0.001232,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C2_WA1_C1-R3
D1,40.0,4.0,0.000835,168.691467,1.582973,5.928974,0.008197,12.175665,2.941506,93.463208,0.000219,0.000281,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C2_WD1_F1-R1
E1,40.0,4.0,0.000835,168.691467,1.582973,5.928974,0.008197,12.175665,2.941506,93.463208,0.000219,0.000281,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C2_WD1_F1-R2
F1,40.0,4.0,0.000835,168.691467,1.582973,5.928974,0.008197,12.175665,2.941506,93.463208,0.000219,0.000281,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C2_WD1_F1-R3
A2,40.0,4.0,0.028070,48.290701,0.050736,3.717661,0.003228,12.521628,2.167364,48.287588,0.000199,0.000102,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C2_WA2_C2-R1
B2,40.0,4.0,0.028070,48.290701,0.050736,3.717661,0.003228,12.521628,2.167364,48.287588,0.000199,0.000102,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C2_WA2_C2-R2
C2,40.0,4.0,0.028070,48.290701,0.050736,3.717661,0.003228,12.521628,2.167364,48.287588,0.000199,0.000102,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C2_WA2_C2-R3


## Process measurement files

Read measurements file:

In [9]:
df_600 = pd.read_excel(user_params['measurement_file'], sheet_name='600', index_col=0)
df_340 = pd.read_excel(user_params['measurement_file'], sheet_name='340', index_col=0)

df_600

,1,2,3,4,5,6,7,8,9,10,11
A,1.5159,1.3528,0.9162,0.3143,0.2451,1.4731,1.1296,0.0957,0.0450,0.0446,0.0446
B,1.4784,1.3420,0.9711,0.3121,0.3347,1.7211,1.0941,0.0887,0.0419,0.0410,0.0415
C,1.4808,1.3551,0.5330,0.3079,0.0861,1.6999,1.1345,0.0851,0.0356,0.0370,0.0353
D,1.4880,0.7454,0.7598,0.7003,0.3414,0.9949,0.4474,0.0524,0.0305,0.0307,0.0300
E,1.4717,0.7518,0.7295,0.6994,0.3459,1.0125,0.3651,0.0432,0.0264,0.0263,0.0262
F,1.3854,0.7257,0.6967,0.7110,0.4445,1.0297,0.4617,0.0447,0.0233,0.0235,0.0229
G,0.0224,0.0227,0.0236,0.0230,0.0231,0.0229,0.0230,0.0231,0.0229,0.0222,0.0223
H,0.0233,0.0235,0.0233,0.0236,0.0235,0.0231,0.0230,0.0230,0.0234,0.0235,0.0228


In [10]:
df_340

,1,2,3,4,5,6,7,8,9,10,11
A,0.6736,0.5827,0.3102,0.3357,0.2608,0.7238,0.7420,0.1748,0.1036,0.1021,0.1005
B,0.5750,0.5881,0.3128,0.3452,0.2745,0.7215,0.7206,0.1693,0.1023,0.0999,0.0981
C,0.5815,0.5871,0.2916,0.3419,0.2668,0.7257,0.7647,0.1652,0.0930,0.0941,0.0911
D,0.5477,0.5591,0.1797,0.3219,0.2767,0.5580,0.2327,0.3611,0.0896,0.0876,0.0853
E,0.5489,0.5783,0.1779,0.3320,0.2883,0.5618,0.2368,0.3270,0.0835,0.0814,0.0806
F,0.5550,0.5718,0.1826,0.3441,0.2915,0.5859,0.2489,0.3423,0.0797,0.0789,0.0765
G,0.0731,0.0752,0.0792,0.0805,0.0819,0.0813,0.0812,0.0814,0.0792,0.0776,0.0753
H,0.0742,0.0747,0.0770,0.0778,0.0767,0.0769,0.0762,0.0772,0.0776,0.0759,0.0748


### Process OD600

Normalize the data to the control well A9 value (water content):

In [11]:
zero_value_600 = df_600.at['A', 9]
df_600.loc[:, df_600.columns] -= zero_value_600
df_600

,1,2,3,4,5,6,7,8,9,10,11
A,1.4709,1.3078,0.8712,0.2693,0.2001,1.4281,1.0846,0.0507,0.0000,-0.0004,-0.0004
B,1.4334,1.2970,0.9261,0.2671,0.2897,1.6761,1.0491,0.0437,-0.0031,-0.0040,-0.0035
C,1.4358,1.3101,0.4880,0.2629,0.0411,1.6549,1.0895,0.0401,-0.0094,-0.0080,-0.0097
D,1.4430,0.7004,0.7148,0.6553,0.2964,0.9499,0.4024,0.0074,-0.0145,-0.0143,-0.0150
E,1.4267,0.7068,0.6845,0.6544,0.3009,0.9675,0.3201,-0.0018,-0.0186,-0.0187,-0.0188
F,1.3404,0.6807,0.6517,0.6660,0.3995,0.9847,0.4167,-0.0003,-0.0217,-0.0215,-0.0221
G,-0.0226,-0.0223,-0.0214,-0.0220,-0.0219,-0.0221,-0.0220,-0.0219,-0.0221,-0.0228,-0.0227
H,-0.0217,-0.0215,-0.0217,-0.0214,-0.0215,-0.0219,-0.0220,-0.0220,-0.0216,-0.0215,-0.0222


Set negative values to zero, multiply the values by 10 to account for 10x dilution and keep only 8 columns and 6 rows:

In [12]:
df_600[df_600 < 0] = 0
df_600.loc[:, df_600.columns] *= 10
df_600 = df_600.iloc[:6,:8]
df_600

,1,2,3,4,5,6,7,8
A,14.709,13.078,8.712,2.693,2.001,14.281,10.846,0.507
B,14.334,12.970,9.261,2.671,2.897,16.761,10.491,0.437
C,14.358,13.101,4.880,2.629,0.411,16.549,10.895,0.401
D,14.430,7.004,7.148,6.553,2.964,9.499,4.024,0.074
E,14.267,7.068,6.845,6.544,3.009,9.675,3.201,0.000
F,13.404,6.807,6.517,6.660,3.995,9.847,4.167,0.000


### Process OD340

Do the same for OD340, except of multiplication by 10, as the OD340 measurements were taken from non-diluted samples:

In [13]:
zero_value_340 = df_340.at['A', 9]
df_340.loc[:, df_340.columns] -= zero_value_340
df_340[df_340 < 0] = 0
df_340 = df_340.iloc[:6,:8]
df_340

,1,2,3,4,5,6,7,8
A,0.5700,0.4791,0.2066,0.2321,0.1572,0.6202,0.6384,0.0712
B,0.4714,0.4845,0.2092,0.2416,0.1709,0.6179,0.6170,0.0657
C,0.4779,0.4835,0.1880,0.2383,0.1632,0.6221,0.6611,0.0616
D,0.4441,0.4555,0.0761,0.2183,0.1731,0.4544,0.1291,0.2575
E,0.4453,0.4747,0.0743,0.2284,0.1847,0.4582,0.1332,0.2234
F,0.4514,0.4682,0.0790,0.2405,0.1879,0.4823,0.1453,0.2387


Transform wide to long format, matching the order of wells in index to the one from `df`:

In [14]:
if column_order:
    indfcn = lambda x: f'{x.name}{int(x["variable"])}'
    # df_600 = df_600.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.melt(ignore_index=False, value_name='OD340')
else:
    indfcn = lambda x: f'{x["variable"]}{(x.name)}'
    # df_600 = df_600.T.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.T.melt(ignore_index=False, value_name='OD340')

# df_600['Well'] = df_600.apply(indfcn, axis=1)
df_340['Well'] = df_340.apply(indfcn, axis=1)

# df_600.index = df_600['Well']
df_340.index = df_340['Well']

# df_600.drop(columns=['variable', 'Well'], inplace=True)
df_340.drop(columns=['variable', 'Well'], inplace=True)

# df_600.head()

Add measurements to the main dataframe:

In [15]:
# measurOD600 = user_params['protocol_name'][0]
measurOD340 = user_params['protocol_name'][1]
# df[measurOD600] = df_600
df[measurOD340] = df_340
df.head(2)


,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name,OD340
Well,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.003982,147.780153,0.263819,1.292187,0.001388,10.224859,1.888788,146.589727,0.000013,0.001232,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C2_WA1_C1-R1,0.5700
B1,40.0,4.0,0.003982,147.780153,0.263819,1.292187,0.001388,10.224859,1.888788,146.589727,0.000013,0.001232,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C2_WA1_C1-R2,0.4714


## Create EDD Experiment Description File

In [16]:
df['Media'] = user_params['media']
df['Part ID'] = user_params['part_id']
df['Culture Volume'] = user_params['culture_volume']
df['Flask Volume'] = user_params['well_volume']
df['Growth Temperature'] = user_params['temperature']
df['Shaking speed'] = user_params['shaking_speed']
# df['Starting OD'] =
# df['Replicate Count'] = 24


In [17]:
# Invalid columns for now in EDD
# df['Humidity[%]'] = user_params['humidity']
# df['Plate'] = user_params['plate']

In [18]:
exp_descr_file = f'{user_params["output_file_path"]}/edd_experiment_description.xlsx'
df[['Line Name',
    'Line Description',
    'Part ID',
    'Media',
    'Culture Volume',
    'Flask Volume',
    'Growth Temperature',
    'Shaking speed',
]].to_excel(exp_descr_file, index=False)

## Create EDD Measurement File

OD600

In [19]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD600}.xlsx'
# df['Measurement Type'] = 'Optical Density'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD600]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD340

In [20]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD340}.xlsx'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD340]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD

In [21]:
measurement_file = f'{user_params["output_file_path"]}/edd_OD.xlsx'
df['Measurement Type'] = measurOD340

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD340]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

In [22]:
df['Kan[g/l]']

Well
A1    0.05
B1    0.05
C1    0.05
D1    0.05
E1    0.05
F1    0.05
A2    0.05
B2    0.05
C2    0.05
D2    0.05
E2    0.05
F2    0.05
A3    0.05
B3    0.05
C3    0.05
D3    0.05
E3    0.05
F3    0.05
A4    0.05
B4    0.05
C4    0.05
D4    0.05
E4    0.05
F4    0.05
A5    0.05
B5    0.05
C5    0.05
D5    0.05
E5    0.05
F5    0.05
A6    0.05
B6    0.05
C6    0.05
D6    0.05
E6    0.05
F6    0.05
A7    0.05
B7    0.05
C7    0.05
D7    0.05
E7    0.05
F7    0.05
A8    0.05
B8    0.05
C8    0.05
D8    0.05
E8    0.05
F8    0.05
Name: Kan[g/l], dtype: float64